In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zeeshan-finbert/__huggingface_repos__.json
/kaggle/input/zeeshan-finbert/finbert_5fold_cv_results_cached.pkl
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/config.json
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/trainer_state.json
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/training_args.bin
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/tokenizer.json
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/tokenizer_config.json
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/scaler.pt
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/scheduler.pt
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/model.safetensors
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/special_tokens_map.json
/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376/optimizer.pt
/kaggle/input/zeeshan-f

In [2]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
MODEL_PATH = "/kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("Model loaded from:", MODEL_PATH)
print("Running on:", device)


2025-12-28 00:22:33.218217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766881353.384936      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766881353.433178      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766881353.805832      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766881353.805871      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766881353.805873      24 computation_placer.cc:177] computation placer alr

Model loaded from: /kaggle/input/zeeshan-finbert/news_headline_sentiment/checkpoint-376
Running on: cuda


In [4]:
label_map = {
    0: "Negative",
    1: "Neutral",
    2: "Positive"
}

def predict_sentiment(headlines):
    if isinstance(headlines, str):
        headlines = [headlines]

    inputs = tokenizer(
        headlines,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

    results = []
    for i, text in enumerate(headlines):
        results.append({
            "headline": text,
            "prediction": label_map[preds[i].item()],
            "confidence": float(probs[i][preds[i]].item())
        })

    return results

In [5]:
test_headlines = [
    "Company reports record quarterly profits",
    "Stock falls amid uncertainty over interest rates",
    "Market remains flat ahead of earnings season"
]

outputs = predict_sentiment(test_headlines)

for o in outputs:
    print(o)


{'headline': 'Company reports record quarterly profits', 'prediction': 'Positive', 'confidence': 0.9997838139533997}
{'headline': 'Stock falls amid uncertainty over interest rates', 'prediction': 'Negative', 'confidence': 0.9288573265075684}
{'headline': 'Market remains flat ahead of earnings season', 'prediction': 'Negative', 'confidence': 0.844763994216919}


In [6]:
import pandas as pd

df = pd.DataFrame({
    "headline": [
        "Oil prices surge after supply cuts",
        "Investors cautious ahead of Fed meeting"
    ]
})

results = predict_sentiment(df["headline"].tolist())
pd.DataFrame(results)

,headline,prediction,confidence
0,Oil prices surge after supply cuts,Negative,0.999122
1,Investors cautious ahead of Fed meeting,Neutral,0.986885
